In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

#Reconstruct results of ETL Notebook
data = pd.read_csv("/content/drive/MyDrive/Learning/Data Science/IBM Advanced DS Certification/train.txt", delimiter= " ", header = None)
data = data.iloc[:, 0:26]
colNames = ["UnitNumber", "TimeCycle", "OpSetting1", "OpSetting2", "OpSetting3"]
sensorCols = ["SensorMeasurement_" + str(x) for x in range(1, 22)]
colNames.extend(sensorCols)
data.columns = colNames
#Compute Lifetime for each TurboFan Engine
usefulLife_df = pd.DataFrame(data["UnitNumber"].value_counts())
usefulLife_df.columns = ["UnitLife"]
usefulLife_df["UnitNumber"] = usefulLife_df.index
usefulLife_df = usefulLife_df.sort_values("UnitNumber")
usefulLife_df = usefulLife_df[["UnitNumber", "UnitLife"]].reset_index(drop = True)
#We'll join the `UnitLife` variable onto the original dataset and computer RUL
data = data.merge(usefulLife_df, how = "left", on = "UnitNumber")
data["RUL"] = data["TimeCycle"] - data["UnitLife"]

#Identify the Operation Configuration for each record
data["OpConfig"] = len(data)*[0]

for i in range(len(data)):
  if data.iloc[i, 4] > 80:
    data.iloc[i, -1] = 1
  
  elif data.iloc[i, 2] < 10:
    data.iloc[i, -1] = 2
  
  elif data.iloc[i, 4] < 20:
    data.iloc[i, -1] = 3
  
  elif data.iloc[i, 2] > 40:
    data.iloc[i, -1] = 4

  elif data.iloc[i, 2] > 30:
    data.iloc[i, -1] = 5
  
  else:
    data.iloc[i, -1] = 6

#Select only the useful features identified in the EDAandFE notebook
data = data[["UnitNumber", "TimeCycle", "OpConfig", "SensorMeasurement_3", "SensorMeasurement_4", "SensorMeasurement_9", "SensorMeasurement_11", "SensorMeasurement_14", "SensorMeasurement_15", "SensorMeasurement_17", "RUL"]].copy()
#Replace OpConfig with dummy variables
data = pd.get_dummies(data, columns = ["OpConfig"], drop_first = False)

##Split into Training, Test, and Safe Data

#Create a list of engines (unique unit numbers)
engines = list(data["UnitNumber"].unique())

#Function to shuffle and split engines into training, test, and safe sets
def split_units(units, random_state = 42, train_size = 0.7, test_size = 0.2):
  units2 = units.copy()
  random.Random(42).shuffle(units2)
  #print(units2)
  trainUnits = units2[:int(train_size*len(units))]
  testUnits = units2[int(train_size*len(units)):int((train_size + test_size)*len(units))]
  safeUnits = units2[int((train_size + test_size)*len(units)):]

  return trainUnits, testUnits, safeUnits

#Split engines into training, test, and safe sets -- ensuring no engines have been lost.
trainUnits, testUnits, safeUnits = split_units(engines, random_state = 42, train_size = 0.7, test_size = 0.2)

#Create training, test, and safe sets corresponding to engines chosen above
trainUnitsData = data[data["UnitNumber"].isin(trainUnits)].copy()
testUnitsData = data[data["UnitNumber"].isin(testUnits)].copy()
safeUnitsData = data[data["UnitNumber"].isin(safeUnits)].copy()

#Print out the head of the training set
trainUnitsData.head()

,UnitNumber,TimeCycle,SensorMeasurement_3,SensorMeasurement_4,SensorMeasurement_9,SensorMeasurement_11,SensorMeasurement_14,SensorMeasurement_15,SensorMeasurement_17,RUL,OpConfig_1,OpConfig_2,OpConfig_3,OpConfig_4,OpConfig_5,OpConfig_6
0,1,1,1499.45,1309.95,8770.20,45.40,8120.83,8.6216,368,-222,0,0,0,0,0,1
1,1,2,1584.55,1403.96,9045.76,47.29,8132.87,8.3907,391,-221,1,0,0,0,0,0
2,1,3,1368.17,1122.49,8343.91,41.92,8063.84,9.3557,334,-220,0,0,0,0,1,0
3,1,4,1488.44,1249.18,8721.53,44.26,8052.30,9.2231,364,-219,0,0,1,0,0,0
4,1,5,1354.48,1124.32,8314.56,41.79,8083.67,9.2986,330,-218,0,0,0,1,0,0


## Preparing for Model Construction

I am using this assignment as an opportunity to explore the construction of an `LSTM` for predicting remaining useful life (`RUL`) for turbofan engines. Each engine is operated until failure under variable operational settings. In our earlier exploratory notebooks, we discovered that there were actually only six different operational configurations rather than a continuum. We've encoded those operational configurations and already constructed dummy columns indicating which configuration the engine is operating under at a given time. Additionally, we've identified seven of the initial 20 sensors which show an association with `RUL`. That is the data set for which we will work with now.

As one more preparatory step, we've split the dataset up into *training*, *test*, and *safe* sets. This was done at the engine (`UnitNumer`) level rather than at the individual record level.

## A Multiple Linear Regression Model

Multiple linear regression models are simple tools for descriptive and predictive tasks. I don't expect a multiple linear regression model to perform well for a remaining useful life (RUL) task because the linear regresion model has no way to prioritize accurate predictions for units nearing end of life (EOL). The model will optimize over all cases it receives -- that is, even though we care most about accurate predictions within the last say 10 or 15 time cycles, the model will also fit to observations with very long remaining useful life. Still, a linear regression model may prove as a useful sanity check and benchmark for a more powerful model.

One thing we could do here to help our linear regression model is to engineer a few new features. We noted that near end of life, many sensor readings showed a climb in their reported values. For this reason, let's engineer a new feature for each sensor which maintains a cumulative average reading. As time goes on, consistent sensor readings which pull this average upwards may indicate impending EOL.

In [2]:
#trainUnitsData.head()
trainUnitsData["SensorMeasurment_3_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_4_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_9_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_11_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_14_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_15_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_17_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData.head()

,UnitNumber,TimeCycle,SensorMeasurement_3,SensorMeasurement_4,SensorMeasurement_9,SensorMeasurement_11,SensorMeasurement_14,SensorMeasurement_15,SensorMeasurement_17,RUL,...,OpConfig_4,OpConfig_5,OpConfig_6,SensorMeasurment_3_CumAvg,SensorMeasurment_4_CumAvg,SensorMeasurment_9_CumAvg,SensorMeasurment_11_CumAvg,SensorMeasurment_14_CumAvg,SensorMeasurment_15_CumAvg,SensorMeasurment_17_CumAvg
0,1,1,1499.45,1309.95,8770.20,45.40,8120.83,8.6216,368,-222,...,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,1584.55,1403.96,9045.76,47.29,8132.87,8.3907,391,-221,...,0,0,0,1499.450000,1309.950,8770.200000,45.4000,8120.830000,8.621600,368.000000
2,1,3,1368.17,1122.49,8343.91,41.92,8063.84,9.3557,334,-220,...,0,1,0,1542.000000,1356.955,8907.980000,46.3450,8126.850000,8.506150,379.500000
3,1,4,1488.44,1249.18,8721.53,44.26,8052.30,9.2231,364,-219,...,0,0,0,1484.056667,1278.800,8719.956667,44.8700,8105.846667,8.789333,364.333333
4,1,5,1354.48,1124.32,8314.56,41.79,8083.67,9.2986,330,-218,...,1,0,0,1485.152500,1271.395,8720.350000,44.7175,8092.460000,8.897775,364.250000


Let's build these features for the *test* and *safe* sets as well.

In [3]:
testUnitsData["SensorMeasurment_3_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_4_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_9_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_11_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_14_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_15_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_17_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())

safeUnitsData["SensorMeasurment_3_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_4_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_9_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_11_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_14_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_15_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_17_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())

Okay, now let's train and assess our linear regressor.

In [4]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1).fillna(0), trainUnitsData["RUL"])

print("Intercept: ", lin_reg.intercept_)
print("Coefficients: ", pd.DataFrame({"Feature" : trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1).columns, "Coefficient" : lin_reg.coef_}))

Intercept:  -7534.16734633543
Coefficients:                         Feature  Coefficient
0                    TimeCycle     0.495564
1          SensorMeasurement_3     0.405731
2          SensorMeasurement_4     0.838880
3          SensorMeasurement_9    -0.084990
4         SensorMeasurement_11    41.773349
5         SensorMeasurement_14     0.087325
6         SensorMeasurement_15   343.315964
7         SensorMeasurement_17     2.188866
8                   OpConfig_1  -171.122826
9                   OpConfig_2   -14.780288
10                  OpConfig_3   -99.835652
11                  OpConfig_4   144.616348
12                  OpConfig_5   155.318322
13                  OpConfig_6   -14.195904
14   SensorMeasurment_3_CumAvg    -2.414589
15   SensorMeasurment_4_CumAvg    -0.294365
16   SensorMeasurment_9_CumAvg     0.550480
17  SensorMeasurment_11_CumAvg    -1.392119
18  SensorMeasurment_14_CumAvg    -0.217167
19  SensorMeasurment_15_CumAvg   -24.502092
20  SensorMeasurment_17_CumAvg 

In [5]:
##Assess Model Performance on test data
preds = lin_reg.predict(testUnitsData.drop(["UnitNumber", "RUL"], axis = 1).fillna(0))
print("RMSE: ", (((preds - testUnitsData["RUL"])**2).mean())**0.5)

RMSE:  35.48781895136142


Let's see if we can do any better with an LSTM network.

## Why LSTM?

I am very comfortable building a variety of regression models, but I have very seldom ventured into deep learning. This is a good opportunity to try it. During the Coursera courses, the instructors discussed LSTM models, which are a type of neural network utilizing recurrent layers that act as memory. These types of networks are well-suited to time series data. Since we are working with streaming sensor data, an LSTM seems like a natural choice.

In [6]:
trainUnitsData = trainUnitsData[['UnitNumber', 'TimeCycle', 'SensorMeasurement_3', 'SensorMeasurement_4',
       'SensorMeasurement_9', 'SensorMeasurement_11', 'SensorMeasurement_14',
       'SensorMeasurement_15', 'SensorMeasurement_17', 'RUL', 'OpConfig_1',
       'OpConfig_2', 'OpConfig_3', 'OpConfig_4', 'OpConfig_5', 'OpConfig_6']]

testUnitsData = testUnitsData[['UnitNumber', 'TimeCycle', 'SensorMeasurement_3', 'SensorMeasurement_4',
       'SensorMeasurement_9', 'SensorMeasurement_11', 'SensorMeasurement_14',
       'SensorMeasurement_15', 'SensorMeasurement_17', 'RUL', 'OpConfig_1',
       'OpConfig_2', 'OpConfig_3', 'OpConfig_4', 'OpConfig_5', 'OpConfig_6']]

safeUnitsData = safeUnitsData[['UnitNumber', 'TimeCycle', 'SensorMeasurement_3', 'SensorMeasurement_4',
       'SensorMeasurement_9', 'SensorMeasurement_11', 'SensorMeasurement_14',
       'SensorMeasurement_15', 'SensorMeasurement_17', 'RUL', 'OpConfig_1',
       'OpConfig_2', 'OpConfig_3', 'OpConfig_4', 'OpConfig_5', 'OpConfig_6']]

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error

trainUnitsData

X_train = trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1)
y_train = trainUnitsData["RUL"]

X_train.head()

,TimeCycle,SensorMeasurement_3,SensorMeasurement_4,SensorMeasurement_9,SensorMeasurement_11,SensorMeasurement_14,SensorMeasurement_15,SensorMeasurement_17,OpConfig_1,OpConfig_2,OpConfig_3,OpConfig_4,OpConfig_5,OpConfig_6
0,1,1499.45,1309.95,8770.20,45.40,8120.83,8.6216,368,0,0,0,0,0,1
1,2,1584.55,1403.96,9045.76,47.29,8132.87,8.3907,391,1,0,0,0,0,0
2,3,1368.17,1122.49,8343.91,41.92,8063.84,9.3557,334,0,0,0,0,1,0
3,4,1488.44,1249.18,8721.53,44.26,8052.30,9.2231,364,0,0,1,0,0,0
4,5,1354.48,1124.32,8314.56,41.79,8083.67,9.2986,330,0,0,0,1,0,0


In [8]:
model = None
model = Sequential()
model.add(Dense(units = 8, input_dim = len(X_train.columns)))
model.add(Dense(units = 16))
model.add(Dense(1))

model.compile(loss = "mse", optimizer = "adam")

In [9]:
model.build()

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 120       
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 281
Trainable params: 281
Non-trainable params: 0
_________________________________________________________________


In [11]:
X_test = testUnitsData.drop(["UnitNumber", "RUL"], axis = 1)
y_test = testUnitsData["RUL"]

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

Epoch 1/5
1018/1018 [==============================] - 3s 2ms/step - loss: 1737875.8750 - val_loss: 3446.5784
Epoch 2/5
1018/1018 [==============================] - 2s 2ms/step - loss: 3633.5520 - val_loss: 3013.7317
Epoch 3/5
1018/1018 [==============================] - 2s 2ms/step - loss: 3033.3755 - val_loss: 2330.0698
Epoch 4/5
1018/1018 [==============================] - 2s 2ms/step - loss: 2457.1028 - val_loss: 1892.1285
Epoch 5/5
1018/1018 [==============================] - 2s 2ms/step - loss: 2090.7039 - val_loss: 2032.8964


In [12]:
preds = model.predict(X_test)[:, 0]

print("RMSE: ", (((y_test - preds)**2).mean())**0.5)

RMSE:  45.0876478845553


The Neural Network used above was a fully connected network with two hidden layers. It did not perform as well as the linear regressor did on the test data, however we did no tuning, chose two hidden layers arbitrarily, and selected the number of neurons at each layer arbitrarily as well. Let's try a true LSTM to close out this project.

In order to do this, we'll need to reshape our dataset. The LSTM will take blocks of chronological observations, so we'll need our data in 3-D tensor format. Let's take blocks of 10 time periods. Note, we should be worried here about ensuring blocks consist of only records from a single engine, but let's just get the network up and running first.

In [13]:
X_train = trainUnitsData

In [14]:
def lstm_data_transform(X_data, y_data, num_steps = 5):
  X, y = list(), list()

  for i in range(X_data.shape[0]):
    end_ix = i + num_steps

    if end_ix >= X_data.shape[0]:
      break

    seq_X = X_data[i:end_ix]
    seq_y = y_data[end_ix]

    X.append(seq_X)
    y.append(seq_y)
  
  x_array = np.array(X)
  y_array = np.array(y)

  return x_array, y_array

trainUnitsData.reset_index(inplace = True, drop = True)

my_train_data, my_train_y = lstm_data_transform(trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1), trainUnitsData["RUL"])

In [20]:
model2 = Sequential()

model2.add(LSTM(10, input_shape = (5, 14)))
model2.add(Dense(32))
model2.add(Dense(1))

model2.compile(loss = "mse", optimizer = "adam")

In [21]:
model2.fit(my_train_data, my_train_y, epochs = 10)

Epoch 1/10
1018/1018 [==============================] - 6s 4ms/step - loss: 11194.8486
Epoch 2/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4881.3652
Epoch 3/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.7349
Epoch 4/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.2471
Epoch 5/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.5127
Epoch 6/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.7188
Epoch 7/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.7983
Epoch 8/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.7144
Epoch 9/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.7490
Epoch 10/10
1018/1018 [==============================] - 4s 4ms/step - loss: 4835.5723


Let's see how this model performs on our test data.

In [22]:
testUnitsData.reset_index(inplace = True, drop = True)

my_test_data, my_test_y = lstm_data_transform(testUnitsData.drop(["UnitNumber", "RUL"], axis = 1), testUnitsData["RUL"])

preds = model2.predict(my_test_data)[:, 0]
preds

array([-111.51099, -111.51099, -111.51099, ..., -111.51099, -111.51099,
       -111.51099], dtype=float32)

In [23]:
print("RMSE: ", (((my_test_y - preds)**2).mean())**0.5)

RMSE:  65.78261265139932


Okay, but we still aren't approaching the performance results achieved with the linear regression model. I saw a video in which failure to use *MinMax scaling* resulted in very poor network performance. Let's try using the scaling here to improve performance.

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_train_data = scaler.fit_transform(trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1))

my_train_data, my_train_y = lstm_data_transform(scaled_train_data, trainUnitsData["RUL"])

model2.fit(my_train_data, my_train_y, epochs = 10)

Epoch 1/10
1018/1018 [==============================] - 4s 4ms/step - loss: 2252.6875
Epoch 2/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1950.3097
Epoch 3/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1862.8223
Epoch 4/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1808.4578
Epoch 5/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1750.5383
Epoch 6/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1685.6449
Epoch 7/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1624.5909
Epoch 8/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1564.9596
Epoch 9/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1541.2974
Epoch 10/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1523.2982


The losses seem to be improved, but we haven't reached anything comparable to our original linear regression model. Still, let's check the performance of our LSTM on the test data.

In [25]:
scaled_test_data = scaler.transform(testUnitsData.drop(["UnitNumber", "RUL"], axis = 1))

my_test_data, my_test_y = lstm_data_transform(scaled_test_data, testUnitsData["RUL"])

preds = model2.predict(my_test_data)[:, 0]

print("RMSE: ", (((my_test_y - preds)**2).mean())**0.5)

RMSE:  36.6306022101016


We've achieved comparable results for both the LSTM network and Linear Regressor. This is interesting -- the linear regressor was given access to the engineered cumulative averages for sensor streams, maybe this was a significant advantage. Let's make one last run, allowing an LSTM access to the cumulative averages.

In [26]:
trainUnitsData["SensorMeasurment_3_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_4_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_9_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_11_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_14_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_15_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
trainUnitsData["SensorMeasurment_17_CumAvg"] = trainUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())

testUnitsData["SensorMeasurment_3_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_4_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_9_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_11_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_14_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_15_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
testUnitsData["SensorMeasurment_17_CumAvg"] = testUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())

safeUnitsData["SensorMeasurment_3_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_3"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_4_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_4"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_9_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_9"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_11_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_11"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_14_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_14"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_15_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_15"].apply(lambda x: x.shift().expanding().mean())
safeUnitsData["SensorMeasurment_17_CumAvg"] = safeUnitsData.groupby("UnitNumber")["SensorMeasurement_17"].apply(lambda x: x.shift().expanding().mean())

trainUnitsData = trainUnitsData.fillna(0)
testUnitsData = testUnitsData.fillna(0)
safeUnitsData = safeUnitsData.fillna(0)

In [27]:
scaler = MinMaxScaler()

scaled_train_data = scaler.fit_transform(trainUnitsData.drop(["UnitNumber", "RUL"], axis = 1))

my_train_data, my_train_y = lstm_data_transform(scaled_train_data, trainUnitsData["RUL"])

In [28]:
my_train_data.shape

(32555, 5, 21)

In [31]:
model3 = Sequential()

model3.add(LSTM(10, input_shape = (5, 21)))
model3.add(Dense(32))
model3.add(Dense(1))

model3.compile(loss = "mse", optimizer = "adam")

In [32]:
model3.fit(my_train_data, my_train_y, epochs = 10)

Epoch 1/10
1018/1018 [==============================] - 7s 4ms/step - loss: 6966.7729
Epoch 2/10
1018/1018 [==============================] - 4s 4ms/step - loss: 3277.2100
Epoch 3/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1929.4250
Epoch 4/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1871.8612
Epoch 5/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1847.2190
Epoch 6/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1826.7637
Epoch 7/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1806.9337
Epoch 8/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1793.6682
Epoch 9/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1776.5826
Epoch 10/10
1018/1018 [==============================] - 4s 4ms/step - loss: 1761.1957


Now let's see the model's performance on the test data.

In [33]:
scaled_test_data = scaler.transform(testUnitsData.drop(["UnitNumber", "RUL"], axis = 1))

my_test_data, my_test_y = lstm_data_transform(scaled_test_data, testUnitsData["RUL"])

preds = model3.predict(my_test_data)[:, 0]

print("RMSE: ", (((my_test_y - preds)**2).mean())**0.5)

RMSE:  38.838560238909416


Interestingly, there was no real improvement here either. Perhaps the cumulative averages were helping the linear regressor gain some memory functionality, and the averages were no advantage to the LSTM because its memory properties already allow it to discover the cumulative averages.

Overall, this was an interesting application. I am surprised that the LSTM network was not able to outperform the linear regressor. Perhaps a different model class altogether may be better suited to this RUL application.